In [1]:
import torch
import torchvision
from data import get_color_distortion
from model import SimCLR, NTXentLoss
from torchvision import transforms
import time
import wandb
from datasets import load_dataset
from pl_bolts.optimizers.lr_scheduler import LinearWarmupCosineAnnealingLR

if torch.cuda.is_available():
    device = torch.device("cuda:0")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print("Training on", device)

# Define a pair transformation that applies the transformation only for one branch of the model
class PairTransform:
    def __init__(self, transform):
        self.transform = transform
        self.original_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    def __call__(self, x):
        return self.original_transform(x), self.transform(x)

def run_experiment(config, transform):
    if config["dataset"] == "CIFAR-10":
        train_dataset = torchvision.datasets.CIFAR10(root='./data', transform=PairTransform(transform), download=True)

    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, drop_last=True)

    model = SimCLR(resnet=config["resnet"], out_dim=config["projection_dim"], projection=config["projection"]).to(device)
    criterion = NTXentLoss(config["batch_size"], device, config["temperature"])
    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
    scheduler = LinearWarmupCosineAnnealingLR(optimizer, warmup_epochs=config["warmup_epochs"], warmup_start_lr=config["learning_rate"] * 1/10, max_epochs=config["epochs"], eta_min=0)

    wandb.init(
        project="SimCLR-augmentation-experiments",
        config=config
    )

    for epoch in range(1, config["epochs"]+1):  # Start epoch numbering from 1
        model.train()
        epoch_start_time = time.time()
        total_loss = 0
        for i, (images, labels) in enumerate(trainloader):
            optimizer.zero_grad()
            h_i, h_j, z_i, z_j = model(images[0].to(device), images[1].to(device))
            loss = criterion(z_i, z_j)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        wandb.log({"lr": scheduler.get_last_lr()}, step=epoch)

        scheduler.step()
        avg_loss = total_loss / len(trainloader)
        # log average loss for epoch to wandb
        wandb.log({"loss": avg_loss}, step=epoch)
        
        print(f'Epoch {epoch} finished in {time.time() - epoch_start_time} seconds, Average Loss: {avg_loss}')

    torch.save(model, f'./models/augmentation_experiments/{config["transform"]}.pth')
    wandb.finish()
    print("Training completed and logs saved.")
    

augmentations = {
    "random_crop": transforms.RandomResizedCrop(32, scale=(0.08, 1.0), ratio=(3.0 / 4.0, 4.0 / 3.0)),
    "color_distortion": get_color_distortion(),
    "gaussian_blur": transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    "shear": transforms.RandomAffine(degrees=0, shear=30),
    "elastic": transforms.ElasticTransform()
}


config = {
    "batch_size": 2048,
    "epochs": 200,
    "learning_rate": 3e-4,
    "weight_decay": 1e-4,
    "momentum": 0.9,
    "projection_dim": 128,
    "projection": "nonlinear",
    "temperature": 0.5,
    "resnet": 18,
    "dataset": "CIFAR-10",
    "optimizer": "AdamW",
    "scheduler": "CosineAnnealingLR",
    "loss": "NT-Xent",
    "warmup_epochs": 10,
}

for name1, transform1 in augmentations.items():
    for name2, transform2 in augmentations.items():
        if name1 == name2:
            config["transform"] = name1
            run_experiment(config, transforms.Compose([
                transform1,
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ]))
        else:
            config["transform"] = name1 + "_and_" + name2
            run_experiment(config, transforms.Compose([
                transform1,
                transform2,
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ]))


/Users/stevan.matovic@onetrust.com/.pyenv/versions/3.10.11/envs/ds/lib/python3.10/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/Users/stevan.matovic@onetrust.com/.pyenv/versions/3.10.11/envs/ds/lib/python3.10/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/Users/stevan.matovic@onetrust.com/.pyenv/versions/3.10.11/envs/ds/lib/python3.10/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": gene

Training on mps
Files already downloaded and verified


/var/folders/q2/t4krq7213hgccsq3l82cs_900000gp/T/ipykernel_59179/1349866135.py:40: UnderReviewWarning: The feature LinearWarmupCosineAnnealingLR is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  scheduler = LinearWarmupCosineAnnealingLR(optimizer, warmup_epochs=config["warmup_epochs"], warmup_start_lr=config["learning_rate"] * 1/10, max_epochs=config["epochs"], eta_min=0)
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: stevan-matovic. Use `wandb login --relogin` to force relogin


Epoch 1 finished in 73.23056888580322 seconds, Average Loss: 7.639720380306244
Epoch 2 finished in 65.41478681564331 seconds, Average Loss: 7.149856885274251
Epoch 3 finished in 67.92625713348389 seconds, Average Loss: 6.979749242464702
Epoch 4 finished in 59.66105008125305 seconds, Average Loss: 6.888481259346008
Epoch 5 finished in 57.16447424888611 seconds, Average Loss: 6.830951015154521
Epoch 6 finished in 56.95068717002869 seconds, Average Loss: 6.792366723219554
Epoch 7 finished in 56.857964754104614 seconds, Average Loss: 6.768590688705444
Epoch 8 finished in 56.52297401428223 seconds, Average Loss: 6.7470338344573975
Epoch 9 finished in 56.642690896987915 seconds, Average Loss: 6.733247558275859
Epoch 10 finished in 56.65328288078308 seconds, Average Loss: 6.714428742726644
Epoch 11 finished in 56.401756048202515 seconds, Average Loss: 6.701370457808177
Epoch 12 finished in 56.55287504196167 seconds, Average Loss: 6.691389342149098
Epoch 13 finished in 56.443639039993286 secon

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


...

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,6.3853


Training completed and logs saved.
Files already downloaded and verified


/var/folders/q2/t4krq7213hgccsq3l82cs_900000gp/T/ipykernel_59179/1349866135.py:40: UnderReviewWarning: The feature LinearWarmupCosineAnnealingLR is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  scheduler = LinearWarmupCosineAnnealingLR(optimizer, warmup_epochs=config["warmup_epochs"], warmup_start_lr=config["learning_rate"] * 1/10, max_epochs=config["epochs"], eta_min=0)


Epoch 1 finished in 84.73026871681213 seconds, Average Loss: 7.719161589940389
Epoch 2 finished in 86.92192268371582 seconds, Average Loss: 7.134414434432983
Epoch 3 finished in 84.03852391242981 seconds, Average Loss: 6.843050042788188
Epoch 4 finished in 83.27088570594788 seconds, Average Loss: 6.725333412488301
Epoch 5 finished in 84.79025411605835 seconds, Average Loss: 6.632374405860901
Epoch 6 finished in 84.05186104774475 seconds, Average Loss: 6.561048169930776
Epoch 7 finished in 83.91335487365723 seconds, Average Loss: 6.5234248240788775
Epoch 8 finished in 84.04877209663391 seconds, Average Loss: 6.501716951529185
Epoch 9 finished in 84.18213820457458 seconds, Average Loss: 6.484394033749898
Epoch 10 finished in 84.07227921485901 seconds, Average Loss: 6.47136648495992
Epoch 11 finished in 84.67604088783264 seconds, Average Loss: 6.460646351178487
Epoch 12 finished in 85.01595902442932 seconds, Average Loss: 6.451785524686177
Epoch 13 finished in 85.30078530311584 seconds, A